In [11]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

In [2]:
training_file = 'TrainingData.pkl'
testing_file= 'TestData.pkl'
# testing_file = ?

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(validation_file, mode='rb') as f:
#     valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = torch.tensor(train['features']), torch.tensor(train['labels'])
# X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = torch.tensor(test['features']), torch.tensor(test['labels'])

In [62]:
transform = transforms.Compose([transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

dataset = datasets.ImageFolder(root='Training', transform=transform)
total = len(dataset)
train_c = 0.85 * len(dataset)
valid_c = 0.15 * len(dataset)

train_data, valid_data = random_split(dataset,[int(train_c),int(valid_c)])
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128,shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=1, shuffle=True)

## Build Model

In [63]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.cnn1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride = 1,padding=0)

        self.cnn2 = nn.Conv2d(in_channels= 6, out_channels= 8, kernel_size= 2, padding=2, stride=1 )
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=1)

        self.fc1 = nn.Linear(16384//2,43)
    

    def forward(self,x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)

        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
    
    def activations(self,x):
        z1 = self.cnn1(x)
        a1 = torch.relu(x)
        out = self.maxpoo1(a1)

        z2 = self.cnn2(out)
        a2 = torch.relu(z2)
        out = out.view(out.size(0),-1)
        return z1,a1, z2, a2, out


In [64]:
model = CNN()
model

CNN(
  (cnn1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(6, 8, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=43, bias=True)
)

In [65]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [66]:
for epoch in range(10):  # 遍历数据集多次
    
    running_loss = 0.0
    total_samples = 0
    running_corrects= 0
    for inputs, labels in trainloader:

        optimizer.zero_grad()  

        outputs = model(inputs)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step() 
        _, preds = torch.max(outputs, 1)
        total_samples += inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


        running_loss += loss.item()
    epoch_acc = running_corrects.double() / len(train_data)
    print(f'Epoch {epoch+1}/{10} - Loss: {running_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    Valid_accuracy = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            z = model(inputs)
            _, preds = torch.max(z, 1)
            Valid_accuracy += (preds == labels).sum().item()
        Valid_accuracy = Valid_accuracy/(len(valid_data))
        print(f'Epoch {epoch+1}/{10} -Valid Accuracy: {Valid_accuracy:.4f}')





Epoch 1/10 - Loss: 357.7369, Accuracy: 0.4357
Epoch 1/10 -Valid Accuracy: 0.6394
Epoch 2/10 - Loss: 153.5428, Accuracy: 0.7460
Epoch 2/10 -Valid Accuracy: 0.8128
Epoch 3/10 - Loss: 88.2249, Accuracy: 0.8625
Epoch 3/10 -Valid Accuracy: 0.8784
Epoch 4/10 - Loss: 56.8006, Accuracy: 0.9157
Epoch 4/10 -Valid Accuracy: 0.8839
Epoch 5/10 - Loss: 42.3911, Accuracy: 0.9381
Epoch 5/10 -Valid Accuracy: 0.9284
Epoch 6/10 - Loss: 30.7364, Accuracy: 0.9565
Epoch 6/10 -Valid Accuracy: 0.9444
Epoch 7/10 - Loss: 24.7286, Accuracy: 0.9645
Epoch 7/10 -Valid Accuracy: 0.9479
Epoch 8/10 - Loss: 23.3123, Accuracy: 0.9643
Epoch 8/10 -Valid Accuracy: 0.9452
Epoch 9/10 - Loss: 16.2635, Accuracy: 0.9772
Epoch 9/10 -Valid Accuracy: 0.9630


KeyboardInterrupt: 